In [301]:
import requests
import _json
from bs4 import BeautifulSoup
import lxml
import re
import pandas as pd

In [28]:
from IPython.core.interactiveshell import InteractiveShell

# Set interactive shell to display all output
InteractiveShell.ast_node_interactivity = "all"


In [326]:
page = requests.get("https://data.un.org/")
soup = BeautifulSoup(page.content, 'lxml')
main_page = soup.find("div", {"class": "CountryList"})
countries = [item.text.strip().split("\n") for item in main_page if len(item.text.strip()) > 1  ]
countries_links = [item["href"] for item in main_page.find_all("a",href=True)]
print((countries))        

[['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Chile', 'China', 'China, Hong Kong SAR', 'China, Macao SAR', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czechia', 'Côte d’Ivoire', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea

In [352]:
all_values_dict = {}
all_countries_data={}
for country,country_link  in enumerate(countries_links[:5]):
    page2 = requests.get(f"https://data.un.org/{country_link}")
    soup2 = BeautifulSoup(page2.content, 'lxml')
    main_page2 = soup2.find_all('tr')
    all_values_dict = {}  # Initialize the dictionary to store the values

    for value in main_page2:
        print(value('td'))
        if len(value('td')) == 4:  # To make sure that the lists don't go out of range
            year_2010 = value('td')[1].text.strip()
            year_2015 = value('td')[2].text.strip()
            year_2021 = value('td')[3].text.strip()
            all_values_dict[value('td')[0].text.strip()] = [year_2010, year_2015, year_2021]
        elif len(value('td')) >= 3:  # Ensure there are at least 3 'td' elements for the else condition
            all_values_dict[value('td')[0].text.strip()] = value('td')[2].text.strip()

    all_countries_data[countries[0][country]] = all_values_dict  # Assign the dictionary to the specified key


[<td style="vertical-align:middle;"></td>, <td><a href="../index.html"><i aria-hidden="true" class="fa fa-chevron-left fa-lg" style="color:#5e75ba;">  </i></a></td>, <td><img alt="AFG" class="medio" height="40vh" id="myImg" src="../../common/flags/afg_flag.png"/></td>, <td class="countrytable">Afghanistan</td>, <td>
</td>, <td><a onclick="show_block()"><i aria-hidden="true" class="fa fa-list-alt fa-lg" style="color:#5e75ba;">  </i></a></td>]
[<td>Region </td>, <td> </td>, <td align="right">Southern Asia<sup></sup></td>]
[<td>Population (000, 2021)</td>, <td> </td>, <td align="right">39 835<sup>a</sup></td>]
[<td>Pop. density (per km2, 2021)</td>, <td> </td>, <td align="right">61<sup>a</sup></td>]
[<td>Capital city </td>, <td> </td>, <td align="right">Kabul<sup></sup></td>]
[<td>Capital city pop. (000, 2021)</td>, <td> </td>, <td align="right">4 114.0<sup>b</sup></td>]
[<td>UN membership date </td>, <td> </td>, <td align="right">19-Nov-46<sup></sup></td>]
[<td>Surface area (km2)</td>, <